In [1]:
# Import the required libraries and dependencies
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Logistic Regression Libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as lr
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Enable the Matplotlib property to allow diagrams to display in Jupyter Notebook
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Read the CSV file into a Pandas DataFrame
# Set the index using the Ticker column
df_feature = pd.read_csv(
    Path("../Data/Feature_Engineering_TD.csv"), 
    index_col="Date"
)

# Review the DataFrame
display(df_feature.head())
display(df_feature.columns)

,Open,High,Low,Close,Adj Close,Volume,Mid_hl,Mid_oc,Mid_hloc,Close_in_range,...,SMA_Signal_8_20Day_Mhloc,high_slope,high_intercept,mid_hl_slope,mid_hl_intercept,mid_oc_slope,mid_oc_intercept,low_slope,low_intercept,trend_10day
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-02 00:00:00-05:00,49.669998,49.700001,49.020000,49.435001,34.867481,3221200,49.360001,49.552500,49.456250,0.610295,...,1,-0.080909,49.741000,-0.065485,49.435667,-0.078106,49.532333,-0.050061,49.130333,0.0
2014-01-03 00:00:00-05:00,49.450001,49.610001,49.325001,49.424999,34.860424,2152600,49.467501,49.437500,49.452500,0.350872,...,1,-0.073939,49.625666,-0.063864,49.366500,-0.060955,49.369500,-0.053788,49.107334,0.0
2014-01-06 00:00:00-05:00,49.610001,49.619999,48.849998,48.994999,34.557133,3970600,49.234999,49.302500,49.268749,0.188312,...,1,-0.047818,49.442998,-0.034409,49.157999,-0.037712,49.194667,-0.021000,48.873000,0.0
2014-01-07 00:00:00-05:00,49.105000,49.389999,48.955002,48.955002,34.528927,2615400,49.172501,49.030001,49.101251,0.000000,...,1,-0.007576,49.197665,-0.002545,48.981999,-0.000864,48.989500,0.002485,48.766334,0.0
2014-01-08 00:00:00-05:00,49.090000,49.154999,48.735001,49.040001,34.588871,3485000,48.945000,49.065001,49.005000,0.726194,...,1,0.039334,48.973665,0.044061,48.765666,0.029030,48.869833,0.048788,48.557667,1.0


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Mid_hl',
       'Mid_oc', 'Mid_hloc', 'Close_in_range', 'Returns_1Day_Moc',
       'Returns_5Day_Moc', 'Returns_30Day_Moc', 'Returns_1Day_Mhloc',
       'Returns_5Day_Mhloc', 'Returns_30Day_Mhloc', 'Std_Dev_10Day_Moc',
       'Std_Dev_30Day_Moc', 'Std_Dev_10Day_Mhloc', 'Std_Dev_30Day_Mhloc',
       'SMA_Returns_8Day_Mhloc', 'SMA_Returns_20Day_Mhloc',
       'SMA_Delta_8_20Day_Mhloc', 'SMA_Signal_8_20Day_Mhloc', 'high_slope',
       'high_intercept', 'mid_hl_slope', 'mid_hl_intercept', 'mid_oc_slope',
       'mid_oc_intercept', 'low_slope', 'low_intercept', 'trend_10day'],
      dtype='object')

In [3]:
df_feature_1 = df_feature
drop_colums_feature = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Mid_hl', 'Mid_oc', 'Mid_hloc',
                        'high_slope','high_intercept', 'mid_hl_slope', 'mid_hl_intercept', 'mid_oc_slope', 'mid_oc_intercept', 'low_slope', 'low_intercept', 'trend_10day']
df_feature_1 = df_feature_1.dropna()
df_feature_1 = df_feature_1.drop(columns = drop_colums_feature)

# Review the DataFrame
print(f"Shape: {df_feature_1.shape}")
print(f"Columns/Features: {df_feature_1.columns}")
display(df_feature_1.head(10))
display(df_feature_1.tail(10))

Shape: (2217, 15)
Columns/Features: Index(['Close_in_range', 'Returns_1Day_Moc', 'Returns_5Day_Moc',
       'Returns_30Day_Moc', 'Returns_1Day_Mhloc', 'Returns_5Day_Mhloc',
       'Returns_30Day_Mhloc', 'Std_Dev_10Day_Moc', 'Std_Dev_30Day_Moc',
       'Std_Dev_10Day_Mhloc', 'Std_Dev_30Day_Mhloc', 'SMA_Returns_8Day_Mhloc',
       'SMA_Returns_20Day_Mhloc', 'SMA_Delta_8_20Day_Mhloc',
       'SMA_Signal_8_20Day_Mhloc'],
      dtype='object')


,Close_in_range,Returns_1Day_Moc,Returns_5Day_Moc,Returns_30Day_Moc,Returns_1Day_Mhloc,Returns_5Day_Mhloc,Returns_30Day_Mhloc,Std_Dev_10Day_Moc,Std_Dev_30Day_Moc,Std_Dev_10Day_Mhloc,Std_Dev_30Day_Mhloc,SMA_Returns_8Day_Mhloc,SMA_Returns_20Day_Mhloc,SMA_Delta_8_20Day_Mhloc,SMA_Signal_8_20Day_Mhloc
Date,,,,,,,,,,,,,,,
2014-02-13 00:00:00-05:00,0.746032,0.000513,0.007329,-0.015287,0.000871,0.008575,-0.013067,0.400593,0.462100,0.419795,0.468387,0.005904,0.005563,0.000341,0
2014-02-14 00:00:00-05:00,0.781814,0.008095,0.008095,-0.005006,0.007119,0.007636,-0.005965,0.450924,0.452834,0.462743,0.457374,0.005019,0.005841,-0.000821,0
2014-02-18 00:00:00-05:00,0.125003,0.004574,0.013641,0.002282,0.005391,0.013847,0.003121,0.509234,0.457470,0.515900,0.463638,0.004901,0.005757,-0.000856,0
2014-02-19 00:00:00-05:00,0.681819,0.000405,0.015509,0.008260,0.000304,0.014519,0.006848,0.469285,0.469792,0.480680,0.474995,0.003681,0.005684,-0.002003,0
2014-02-20 00:00:00-05:00,0.714286,0.005765,0.019479,0.013350,0.005866,0.019685,0.014743,0.412498,0.495594,0.416733,0.502467,0.003424,0.005789,-0.002365,0
2014-02-21 00:00:00-05:00,0.000000,0.001006,0.019982,0.013594,0.001357,0.020180,0.015603,0.426414,0.521533,0.426693,0.531072,0.003222,0.005725,-0.002502,0
2014-02-24 00:00:00-05:00,0.219994,-0.004521,0.007217,0.011690,-0.004669,0.008239,0.012435,0.426512,0.535538,0.426060,0.546005,0.003889,0.005673,-0.001783,0
2014-02-25 00:00:00-05:00,0.416668,0.000101,0.002732,0.013292,-0.001261,0.001568,0.011701,0.405793,0.548802,0.399813,0.557421,0.003961,0.005253,-0.001292,0
2014-02-26 00:00:00-05:00,0.372545,0.000202,0.002529,0.014638,0.000455,0.001719,0.013611,0.351291,0.561046,0.351934,0.568608,0.003979,0.005153,-0.001174,0


,Close_in_range,Returns_1Day_Moc,Returns_5Day_Moc,Returns_30Day_Moc,Returns_1Day_Mhloc,Returns_5Day_Mhloc,Returns_30Day_Mhloc,Std_Dev_10Day_Moc,Std_Dev_30Day_Moc,Std_Dev_10Day_Mhloc,Std_Dev_30Day_Mhloc,SMA_Returns_8Day_Mhloc,SMA_Returns_20Day_Mhloc,SMA_Delta_8_20Day_Mhloc,SMA_Signal_8_20Day_Mhloc
Date,,,,,,,,,,,,,,,
2022-12-01 00:00:00-05:00,1.000000,0.017864,0.002584,0.069196,0.018452,0.001127,0.068017,0.667769,1.497654,0.688912,1.478372,0.008309,0.007982,0.000327,0
2022-12-02 00:00:00-05:00,0.632353,0.009598,0.011540,0.075673,0.011392,0.011753,0.076554,0.792560,1.524363,0.834721,1.510164,0.008315,0.007827,0.000488,0
2022-12-05 00:00:00-05:00,0.394906,-0.004618,0.014113,0.057177,-0.006080,0.014629,0.056366,0.766215,1.560394,0.815069,1.542418,0.008604,0.007898,0.000705,0
2022-12-06 00:00:00-05:00,0.265116,-0.008514,0.011469,0.047874,-0.008684,0.011620,0.047012,0.754066,1.541571,0.805332,1.519742,0.009248,0.008232,0.001016,0
2022-12-07 00:00:00-05:00,0.059829,-0.012220,0.001786,0.031737,-0.011019,0.003718,0.029566,0.828235,1.495559,0.865855,1.483686,0.010173,0.008606,0.001567,0
2022-12-08 00:00:00-05:00,0.922223,-0.000446,-0.016234,0.027261,-0.001755,-0.016196,0.024674,0.881826,1.458339,0.918729,1.451464,0.009934,0.007955,0.001979,0
2022-12-09 00:00:00-05:00,0.628866,0.006356,-0.019394,0.030956,0.008287,-0.019216,0.032575,0.874129,1.435758,0.901596,1.429438,0.009990,0.008053,0.001937,0
2022-12-12 00:00:00-05:00,0.906477,0.006371,-0.008568,0.038710,0.004068,-0.009203,0.037339,0.877703,1.418274,0.901029,1.408431,0.010069,0.007726,0.002343,0
2022-12-13 00:00:00-05:00,0.110169,-0.007487,-0.007541,0.026473,-0.006808,-0.007327,0.025702,0.851466,1.392091,0.871695,1.382180,0.008084,0.007986,0.000098,0


In [4]:
# Read the CSV file into a Pandas DataFrame
# Set the index using the Ticker column
df_target = pd.read_csv(
    Path("../Data/Feature_Engineering_RY.csv"), 
    index_col="Date"
)

# Review the DataFrame
df_target.head()

,Open,High,Low,Close,Adj Close,Volume,Mid_hl,Mid_oc,Mid_hloc,Close_in_range,...,SMA_Signal_8_20Day_Mhloc,high_slope,high_intercept,mid_hl_slope,mid_hl_intercept,mid_oc_slope,mid_oc_intercept,low_slope,low_intercept,trend_10day
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-02 00:00:00-05:00,71.300003,71.639999,70.779999,71.589996,49.878262,1576100,71.209999,71.445000,71.327499,0.941857,...,1,-0.020970,71.529333,-0.000273,71.150000,-0.015939,71.263666,0.020424,70.770666,0.0
2014-01-03 00:00:00-05:00,71.370003,71.459999,71.110001,71.389999,49.738911,1243400,71.285000,71.380001,71.332500,0.800000,...,1,-0.008849,71.434667,0.002273,71.127000,0.002758,71.135333,0.013394,70.819334,1.0
2014-01-06 00:00:00-05:00,71.449997,71.500000,70.610001,70.699997,49.258179,2737700,71.055000,71.074997,71.064999,0.101120,...,1,0.031758,71.269334,0.036273,70.970001,0.037121,70.958332,0.040788,70.670668,1.0
2014-01-07 00:00:00-05:00,71.000000,71.440002,70.970001,70.970001,49.446289,1374100,71.205002,70.985001,71.095001,0.000000,...,1,0.112727,70.960001,0.093576,70.785334,0.096606,70.759667,0.074424,70.610667,1.0
2014-01-08 00:00:00-05:00,70.959999,71.180000,70.669998,71.000000,49.467197,1293700,70.924999,70.980000,70.952499,0.647060,...,1,0.199939,70.671333,0.184333,70.473667,0.166152,70.572666,0.168727,70.276000,1.0


In [5]:
df_target_1 = df_target[['trend_10day']]

# Review the DataFrame
print(f"Shape: {df_target_1.shape}")
print(f"Columns/Features: {df_target_1.columns}")
display(df_target_1.head(10))
display(df_target_1.tail(10))

Shape: (2258, 1)
Columns/Features: Index(['trend_10day'], dtype='object')


,trend_10day
Date,
2014-01-02 00:00:00-05:00,0.0
2014-01-03 00:00:00-05:00,1.0
2014-01-06 00:00:00-05:00,1.0
2014-01-07 00:00:00-05:00,1.0
2014-01-08 00:00:00-05:00,1.0
2014-01-09 00:00:00-05:00,1.0
2014-01-10 00:00:00-05:00,1.0
2014-01-13 00:00:00-05:00,1.0
2014-01-14 00:00:00-05:00,0.0


,trend_10day
Date,
2022-12-15 00:00:00-05:00,NaN
2022-12-16 00:00:00-05:00,NaN
2022-12-19 00:00:00-05:00,NaN
2022-12-20 00:00:00-05:00,NaN
2022-12-21 00:00:00-05:00,NaN
2022-12-22 00:00:00-05:00,NaN
2022-12-23 00:00:00-05:00,NaN
2022-12-28 00:00:00-05:00,NaN
2022-12-29 00:00:00-05:00,NaN


In [6]:
df_model = pd.concat([df_feature_1, df_target_1], axis="columns", join="inner")

# Review the DataFrame
print(f"Shape: {df_model.shape}")
print(f"Columns/Features: {df_model.columns}")
display(df_model.head(10))
display(df_model.tail(10))

Shape: (2217, 16)
Columns/Features: Index(['Close_in_range', 'Returns_1Day_Moc', 'Returns_5Day_Moc',
       'Returns_30Day_Moc', 'Returns_1Day_Mhloc', 'Returns_5Day_Mhloc',
       'Returns_30Day_Mhloc', 'Std_Dev_10Day_Moc', 'Std_Dev_30Day_Moc',
       'Std_Dev_10Day_Mhloc', 'Std_Dev_30Day_Mhloc', 'SMA_Returns_8Day_Mhloc',
       'SMA_Returns_20Day_Mhloc', 'SMA_Delta_8_20Day_Mhloc',
       'SMA_Signal_8_20Day_Mhloc', 'trend_10day'],
      dtype='object')


,Close_in_range,Returns_1Day_Moc,Returns_5Day_Moc,Returns_30Day_Moc,Returns_1Day_Mhloc,Returns_5Day_Mhloc,Returns_30Day_Mhloc,Std_Dev_10Day_Moc,Std_Dev_30Day_Moc,Std_Dev_10Day_Mhloc,Std_Dev_30Day_Mhloc,SMA_Returns_8Day_Mhloc,SMA_Returns_20Day_Mhloc,SMA_Delta_8_20Day_Mhloc,SMA_Signal_8_20Day_Mhloc,trend_10day
Date,,,,,,,,,,,,,,,,
2014-02-13 00:00:00-05:00,0.746032,0.000513,0.007329,-0.015287,0.000871,0.008575,-0.013067,0.400593,0.462100,0.419795,0.468387,0.005904,0.005563,0.000341,0,1.0
2014-02-14 00:00:00-05:00,0.781814,0.008095,0.008095,-0.005006,0.007119,0.007636,-0.005965,0.450924,0.452834,0.462743,0.457374,0.005019,0.005841,-0.000821,0,1.0
2014-02-18 00:00:00-05:00,0.125003,0.004574,0.013641,0.002282,0.005391,0.013847,0.003121,0.509234,0.457470,0.515900,0.463638,0.004901,0.005757,-0.000856,0,1.0
2014-02-19 00:00:00-05:00,0.681819,0.000405,0.015509,0.008260,0.000304,0.014519,0.006848,0.469285,0.469792,0.480680,0.474995,0.003681,0.005684,-0.002003,0,0.0
2014-02-20 00:00:00-05:00,0.714286,0.005765,0.019479,0.013350,0.005866,0.019685,0.014743,0.412498,0.495594,0.416733,0.502467,0.003424,0.005789,-0.002365,0,0.0
2014-02-21 00:00:00-05:00,0.000000,0.001006,0.019982,0.013594,0.001357,0.020180,0.015603,0.426414,0.521533,0.426693,0.531072,0.003222,0.005725,-0.002502,0,0.0
2014-02-24 00:00:00-05:00,0.219994,-0.004521,0.007217,0.011690,-0.004669,0.008239,0.012435,0.426512,0.535538,0.426060,0.546005,0.003889,0.005673,-0.001783,0,0.0
2014-02-25 00:00:00-05:00,0.416668,0.000101,0.002732,0.013292,-0.001261,0.001568,0.011701,0.405793,0.548802,0.399813,0.557421,0.003961,0.005253,-0.001292,0,0.0
2014-02-26 00:00:00-05:00,0.372545,0.000202,0.002529,0.014638,0.000455,0.001719,0.013611,0.351291,0.561046,0.351934,0.568608,0.003979,0.005153,-0.001174,0,0.0


,Close_in_range,Returns_1Day_Moc,Returns_5Day_Moc,Returns_30Day_Moc,Returns_1Day_Mhloc,Returns_5Day_Mhloc,Returns_30Day_Mhloc,Std_Dev_10Day_Moc,Std_Dev_30Day_Moc,Std_Dev_10Day_Mhloc,Std_Dev_30Day_Mhloc,SMA_Returns_8Day_Mhloc,SMA_Returns_20Day_Mhloc,SMA_Delta_8_20Day_Mhloc,SMA_Signal_8_20Day_Mhloc,trend_10day
Date,,,,,,,,,,,,,,,,
2022-12-01 00:00:00-05:00,1.000000,0.017864,0.002584,0.069196,0.018452,0.001127,0.068017,0.667769,1.497654,0.688912,1.478372,0.008309,0.007982,0.000327,0,0.0
2022-12-02 00:00:00-05:00,0.632353,0.009598,0.011540,0.075673,0.011392,0.011753,0.076554,0.792560,1.524363,0.834721,1.510164,0.008315,0.007827,0.000488,0,0.0
2022-12-05 00:00:00-05:00,0.394906,-0.004618,0.014113,0.057177,-0.006080,0.014629,0.056366,0.766215,1.560394,0.815069,1.542418,0.008604,0.007898,0.000705,0,0.0
2022-12-06 00:00:00-05:00,0.265116,-0.008514,0.011469,0.047874,-0.008684,0.011620,0.047012,0.754066,1.541571,0.805332,1.519742,0.009248,0.008232,0.001016,0,0.0
2022-12-07 00:00:00-05:00,0.059829,-0.012220,0.001786,0.031737,-0.011019,0.003718,0.029566,0.828235,1.495559,0.865855,1.483686,0.010173,0.008606,0.001567,0,0.0
2022-12-08 00:00:00-05:00,0.922223,-0.000446,-0.016234,0.027261,-0.001755,-0.016196,0.024674,0.881826,1.458339,0.918729,1.451464,0.009934,0.007955,0.001979,0,0.0
2022-12-09 00:00:00-05:00,0.628866,0.006356,-0.019394,0.030956,0.008287,-0.019216,0.032575,0.874129,1.435758,0.901596,1.429438,0.009990,0.008053,0.001937,0,0.0
2022-12-12 00:00:00-05:00,0.906477,0.006371,-0.008568,0.038710,0.004068,-0.009203,0.037339,0.877703,1.418274,0.901029,1.408431,0.010069,0.007726,0.002343,0,0.0
2022-12-13 00:00:00-05:00,0.110169,-0.007487,-0.007541,0.026473,-0.006808,-0.007327,0.025702,0.851466,1.392091,0.871695,1.382180,0.008084,0.007986,0.000098,0,0.0


In [7]:
# Train Test Split
df_model = df_model.dropna()

# Train 2014 through 2017
df_train = df_model.loc['2014-01-01':'2017-12-31']

# Test 2018
df_test = df_model.loc['2018-01-01':'2018-12-31']


In [25]:
!pip install --pre pycaret

In [31]:
import sklearn
from pycaret.datasets import get_data
from pycaret.classification import *


In [32]:
data = df_train
data.reset_index(inplace=True, drop=True)

data_unseen = df_test
data_unseen.reset_index(inplace=True, drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))


Data for Modeling: (972, 16)
Unseen Data For Predictions: (250, 16)


In [33]:
exp_clf101 = setup(data = data, target = 'trend_10day') 

IntProgress(value=0, description='Processing: ', max=13)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:32:34
Status,. . . . . . . . . . . . . . . . . .,Preparing Data for Modeling
ETC,. . . . . . . . . . . . . . . . . .,Calculating ETC


ValueError: Sklearn version error

In [30]:
sklearn.__version__



NameError: name 'sklearn' is not defined